In [1]:
import glob
import time
import os
import pandas as pd
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
from argparse import ArgumentParser, Namespace
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from itertools import chain
from tqdm import tqdm
import copy
import shutil
import pickle

I assume that the candidate generation and feature genration has already be run on the training and dev tables

In [2]:
es_url = 'http://ckg07:9200'
es_index = 'wikidatadwd-augmented'

# Input Paths

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-train-canonical/
train_path = "/Users/amandeep/Github/table-linker/data/SemTabR4_T2dv2/train-canonical"

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/canonical-with-context/t2dv2-dev-canonical/
dev_path = "/Users/amandeep/Github/table-linker/data/t2dv2/t2dv2-dev-canonical"

# GDrive Path: /table-linker-dataset/2019-iswc_challenge_data/t2dv2/ground_truth/Xinting_GT_csv
ground_truth_files = "/Users/amandeep/Github/table-linker/data/SemTabR4_T2dv2/GT"

# can be downloaded from https://github.com/usc-isi-i2/table-linker-pipelines/blob/main/table-linker-full-pipeline/models/weighted_lr.pkl
classifier_model_path = '/Users/amandeep/Github/table-linker-pipelines/table-linker-full-pipeline/models/weighted_lr.pkl'


# OUTPUT PATHS
output_path = "/Users/amandeep/Github/table-linker/data/SemTabR4_T2dv2/table-linker"
train_output_path = f'{output_path}/train1-output'
dev_output_path = f'{output_path}/dev-output'

# increase version to create a new folder for an experiment
PREVIOUS_VERSION = "v10_342_files"
VERSION = "v12"

train_candidate_path = f'{train_output_path}/{VERSION}/candidates'
train_feature_path = f'{train_output_path}/{VERSION}/features'
train_missing_candidates_path = f'{train_output_path}/{VERSION}/train_missing_candidates_path'

dev_candidate_path = f'{dev_output_path}/{VERSION}/candidates'
dev_feature_path = f'{dev_output_path}/{VERSION}/features'
dev_output_predictions = f'{dev_output_path}/{VERSION}/dev_predictions'
dev_predictions_top_k = f'{dev_output_path}/{VERSION}/dev_predictions_top_k'
dev_colorized_path = f'{dev_output_path}/{VERSION}/dev_predictions_colorized'
dev_metrics_path = f'{dev_output_path}/{VERSION}/dev_predictions_metrics'
dev_missing_candidates_path = f'{dev_output_path}/{VERSION}/dev_missing_candidates_path'

aux_field = 'graph_embedding_complex,class_count,property_count,context'


train_prop_count = f'{train_output_path}/{VERSION}/train_prop_count' 
train_class_count = f'{train_output_path}/{VERSION}/train_class_count'
train_context_path = f'{train_output_path}/{VERSION}/train_context'
train_graph_embedding = f'{train_output_path}/{VERSION}/train_graph_embedding'

dev_prop_count = f'{dev_output_path}/{VERSION}/dev_prop_count'
dev_class_count = f'{dev_output_path}/{VERSION}/dev_class_count'
dev_context_path = f'{dev_output_path}/{VERSION}/dev_context'
dev_graph_embedding = f'{dev_output_path}/{VERSION}/dev_graph_embedding'

temp_dir = f'{output_path}/temp'

pos_output = f'{temp_dir}/training_data/pos_features.pkl'
neg_output = f'{temp_dir}/training_data/neg_features.pkl'
min_max_scaler_path = f'{temp_dir}/training_data/normalization_factor.pkl'

final_score_column = 'siamese_prediction'

model_save_path = f'{dev_output_path}/{VERSION}/saved_models'
best_model_path = ''

copy_candidates_from_previous_version = False

In [3]:
!mkdir -p "$temp_dir"

!mkdir -p "$train_prop_count"
!mkdir -p "$dev_prop_count"
!mkdir -p "$train_class_count"
!mkdir -p "$dev_class_count"
!mkdir -p "$train_graph_embedding"
!mkdir -p "$dev_graph_embedding"
!mkdir -p "$train_context_path"
!mkdir -p "$dev_context_path"

!mkdir -p "$train_candidate_path"
!mkdir -p "$dev_candidate_path"

!mkdir -p "$train_feature_path"
!mkdir -p "$dev_feature_path"

!mkdir -p "$temp_dir/training_data"
!mkdir -p "$dev_output_predictions"
!mkdir -p "$model_save_path"
!mkdir -p "$dev_predictions_top_k"
!mkdir -p "$dev_colorized_path"
!mkdir -p "$dev_metrics_path"
!mkdir -p "$dev_missing_candidates_path"
!mkdir -p "$train_missing_candidates_path"

In [4]:
if copy_candidates_from_previous_version:
    !cp $dev_output_path/$PREVIOUS_VERSION/candidates/*csv $dev_output_path/$VERSION/candidates
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_prop_count/* $dev_output_path/$VERSION/dev_prop_count
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_class_count/* $dev_output_path/$VERSION/dev_class_count
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_context/* $dev_output_path/$VERSION/dev_context
    !cp $dev_output_path/$PREVIOUS_VERSION/dev_graph_embedding/* $dev_output_path/$VERSION/dev_graph_embedding
    
    !cp $train_output_path/$PREVIOUS_VERSION/candidates/*csv $train_output_path/$VERSION/candidates
    !cp $train_output_path/$PREVIOUS_VERSION/train_prop_count/* $train_output_path/$VERSION/train_prop_count
    !cp $train_output_path/$PREVIOUS_VERSION/train_class_count/* $train_output_path/$VERSION/train_class_count
    !cp $train_output_path/$PREVIOUS_VERSION/train_context/* $train_output_path/$VERSION/train_context
    !cp $train_output_path/$PREVIOUS_VERSION/train_graph_embedding/* $train_output_path/$VERSION/train_graph_embedding

In [7]:
features = ['pagerank','retrieval_score','monge_elkan','monge_elkan_aliases','des_cont_jaccard',
            'jaro_winkler','levenshtein','singleton','num_char','num_tokens',
           'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score',
           'lof-graph-embedding-score', 'lof-reciprocal-rank', 'context_score']

In [8]:
classifier_features = ['aligned_pagerank', 'smallest_qnode_number', 'monge_elkan', 'des_cont_jaccard_normalized']

## Candidate Generation

In [7]:
def candidate_generation(path, gt_path, output_path, class_count_path, prop_count_path, context_path, graph_embedding):
    file_list = glob.glob(path + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        st = time.time()
        filename = file.split('/')[-1]
#         print(f"{filename}: {i+1} of {len(file_list)}")
        gt_file = f"{ground_truth_files}/{filename}"
        output_file = f"{output_path}/{filename}"
        
        !tl clean -c label -o label_clean "$file" / \
        --url $es_url --index $es_index \
        get-fuzzy-augmented-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" / \
        --url $es_url --index $es_index \
        get-exact-matches -c label_clean \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" / \
        / get-kgtk-search-matches -c label_clean  \
        --auxiliary-fields {aux_field} \
        --auxiliary-folder "$temp_dir" \
        / ground-truth-labeler --gt-file "$gt_file" > "$output_file"
        
        for field in aux_field.split(','):
            aux_list = []
            for f in glob.glob(f'{temp_dir}/*{field}.tsv'):
                aux_list.append(pd.read_csv(f, sep='\t', dtype=object))
            aux_df = pd.concat(aux_list).drop_duplicates(subset=['qnode'])
            if field == 'class_count':
                class_count_file = f"{class_count_path}/{filename.strip('.csv')}_class_count.tsv"
                aux_df.to_csv(class_count_file, sep='\t', index=False)
            elif field == 'property_count':
                prop_count_file = f"{prop_count_path}/{filename.strip('.csv')}_prop_count.tsv"
                aux_df.to_csv(prop_count_file, sep='\t', index=False)
            elif field == 'context':
                context_file = f"{context_path}/{filename.strip('.csv')}_context.tsv"
                aux_df.to_csv(context_file, sep='\t', index=False)
            else:
                graph_embedding_file = f"{graph_embedding}/{filename.strip('.csv')}_graph_embedding_complex.tsv"
                aux_df.to_csv(graph_embedding_file, sep='\t', index=False)
        
#         print(time.time() - st)


In [8]:
if not copy_candidates_from_previous_version:
    candidate_generation(train_path, ground_truth_files, train_candidate_path, train_class_count, train_prop_count, train_context_path,train_graph_embedding)

0it [00:00, ?it/s]

clean Time: 0.0038008689880371094s
get-fuzzy-augmented-matches Time: 5.1828460693359375s
get-exact-matches Time: 0.6038298606872559s
get-kgtk-search-matches Time: 4.174046993255615s
ground-truth-labeler Time: 0.09036588668823242s


1it [00:19, 19.19s/it]

clean Time: 0.0024907588958740234s
get-fuzzy-augmented-matches Time: 3.837916135787964s
get-exact-matches Time: 0.5225467681884766s
get-kgtk-search-matches Time: 3.588423013687134s
ground-truth-labeler Time: 0.2516970634460449s


2it [00:36, 17.93s/it]

...

340it [2:19:22, 35.61s/it]

clean Time: 0.004926919937133789s
get-fuzzy-augmented-matches Time: 5.597031116485596s
get-exact-matches Time: 0.2886998653411865s
get-kgtk-search-matches Time: 8.571640014648438s
ground-truth-labeler Time: 0.10552978515625s


341it [2:19:45, 31.88s/it]

clean Time: 0.018839120864868164s
get-fuzzy-augmented-matches Time: 21.067628860473633s
get-exact-matches Time: 2.4339699745178223s
get-kgtk-search-matches Time: 41.69972801208496s
ground-truth-labeler Time: 0.547119140625s


342it [2:21:06, 24.76s/it]


In [10]:
if not copy_candidates_from_previous_version:
    candidate_generation(dev_path, ground_truth_files, dev_candidate_path, dev_class_count, dev_prop_count, dev_context_path, dev_graph_embedding)

0it [00:00, ?it/s]

clean Time: 0.005154609680175781s
get-fuzzy-augmented-matches Time: 9.847553014755249s
get-exact-matches Time: 1.1910877227783203s
get-kgtk-search-matches Time: 19.845626831054688s
ground-truth-labeler Time: 0.20462584495544434s


1it [00:40, 40.76s/it]

clean Time: 0.0016167163848876953s
get-fuzzy-augmented-matches Time: 3.5746359825134277s
get-exact-matches Time: 0.3863821029663086s
get-kgtk-search-matches Time: 5.325264930725098s
ground-truth-labeler Time: 0.07149434089660645s


2it [00:57, 26.70s/it]

...

7it [03:31, 33.12s/it]

clean Time: 0.00979304313659668s
get-fuzzy-augmented-matches Time: 14.7984139919281s
get-exact-matches Time: 1.862421989440918s
get-kgtk-search-matches Time: 41.22956299781799s
ground-truth-labeler Time: 0.49227309226989746s


8it [04:41, 44.89s/it]

clean Time: 0.003983974456787109s
get-fuzzy-augmented-matches Time: 5.238208055496216s
get-exact-matches Time: 0.5212647914886475s
get-kgtk-search-matches Time: 14.30022406578064s
ground-truth-labeler Time: 0.14105892181396484s


9it [05:11, 34.66s/it]


## Feature Generation

In [10]:
def feature_generation(candidate_dir, embedding_dir, class_count_dir, property_count_dir, context_path, output_path):
    file_list = glob.glob(candidate_dir + '/*.csv')
    for i, file in tqdm(enumerate(file_list)):
        filename = file.split('/')[-1]
#         print(f"{filename}: {i+1} of {len(file_list)}")
        embedding_file = f"{embedding_dir}/{filename.strip('.csv')}_graph_embedding_complex.tsv"
        class_count_file = f"{class_count_dir}/{filename.strip('.csv')}_class_count.tsv"
        property_count_file = f"{property_count_dir}/{filename.strip('.csv')}_prop_count.tsv"
        context_file = f"{context_path}/{filename.strip('.csv')}_context.tsv"
        output_file = f"{output_path}/{filename}"
        classifier_features_str = ",".join(classifier_features)
        !tl align-page-rank $file \
            / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -o monge_elkan \
            / string-similarity -i --method symmetric_monge_elkan:tokenizer=word -c label_clean kg_aliases -o monge_elkan_aliases \
            / string-similarity -i --method jaro_winkler -o jaro_winkler \
            / string-similarity -i --method levenshtein -o levenshtein \
            / string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
            / normalize-scores -c des_cont_jaccard / smallest-qnode-number \
            / mosaic-features -c kg_labels --num-char --num-tokens \
            / create-singleton-feature -o singleton \
            / vote-by-classifier  \
            --prob-threshold 0.995 \
            --features $classifier_features_str \
            --model $classifier_model_path \
            / score-using-embedding \
            --column-vector-strategy centroid-of-lof \
            --lof-strategy ems-mv \
            -o lof-graph-embedding-score \
            --embedding-file $embedding_file \
            / generate-reciprocal-rank  \
            -c lof-graph-embedding-score \
            -o lof-reciprocal-rank \
            / compute-tf-idf  \
            --feature-file $class_count_file \
            --feature-name class_count \
            --singleton-column singleton \
            -o lof_class_count_tf_idf_score \
            / compute-tf-idf \
            --feature-file $property_count_file \
            --feature-name property_count \
            --singleton-column singleton \
            -o lof_property_count_tf_idf_score \
            / context-match --context-file $context_file \
            -o context_score \
            > $output_file

In [ ]:
feature_generation(train_candidate_path, train_graph_embedding, train_class_count, train_prop_count, train_context_path, train_feature_path)

0it [00:00, ?it/s]

align-page-rank Time: 0.22191905975341797s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 4.466212749481201s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 12.080178022384644s
string-similarity-['jaro_winkler'] Time: 0.495136022567749s
string-similarity-['levenshtein'] Time: 5.341928958892822s
string-similarity-['jaccard:tokenizer=word'] Time: 0.05250382423400879s
normalize-scores-des_cont_jaccard Time: 0.026294946670532227s
smallest-qnode-number Time: 0.21240901947021484s
mosaic-features Time: 0.010735750198364258s
creat-singleton-feature Time: 0.16389822959899902s
vote-by-classifier Time: 2.3569889068603516s
Qnodes to lookup: 3148
Qnodes from file: 3089
Outlier removal generates 4 lof-voted candidates
Outlier removal generates 21 lof-voted candidates
score-using-embedding Time: 36.75325894355774s
generate-reciprocal-rank-lof-graph-embedding-score Time: 0.2624509334564209s
compute-tf-idf-class_count Time: 38.3315007686615s
compute-tf-idf-property_co

1it [01:15, 75.37s/it]

align-page-rank Time: 0.14556026458740234s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 3.3670871257781982s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 2.0737719535827637s
string-similarity-['jaro_winkler'] Time: 0.7165331840515137s
string-similarity-['levenshtein'] Time: 5.139523029327393s
string-similarity-['jaccard:tokenizer=word'] Time: 0.029052257537841797s
normalize-scores-des_cont_jaccard Time: 0.01286005973815918s
smallest-qnode-number Time: 0.08622193336486816s
mosaic-features Time: 0.006398200988769531s
creat-singleton-feature Time: 0.39304304122924805s
vote-by-classifier Time: 0.9565598964691162s
Qnodes to lookup: 436
Qnodes from file: 436
Outlier removal generates 11 lof-voted candidates
score-using-embedding Time: 22.45358395576477s
generate-reciprocal-rank-lof-graph-embedding-score Time: 0.11409997940063477s
compute-tf-idf-class_count Time: 24.281742811203003s
compute-tf-idf-property_count Time: 23.49156093597412s
context-match Tim

2it [01:51, 52.32s/it]

...

align-page-rank Time: 0.15169000625610352s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 0.3960390090942383s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 0.8255980014801025s
string-similarity-['jaro_winkler'] Time: 0.12698101997375488s
string-similarity-['levenshtein'] Time: 0.43778085708618164s
string-similarity-['jaccard:tokenizer=word'] Time: 0.03637337684631348s
normalize-scores-des_cont_jaccard Time: 0.013297796249389648s
smallest-qnode-number Time: 0.0926821231842041s
mosaic-features Time: 0.009131908416748047s
creat-singleton-feature Time: 0.06321382522583008s
vote-by-classifier Time: 0.40155696868896484s
Qnodes to lookup: 1403
Qnodes from file: 1387
Outlier removal generates 5 lof-voted candidates
score-using-embedding Time: 9.314464092254639s
generate-reciprocal-rank-lof-graph-embedding-score Time: 0.11962318420410156s
compute-tf-idf-class_count Time: 10.550337076187134s
compute-tf-idf-property_count Time: 10.708681106567383s
context-matc

340it [7:14:05, 118.81s/it]

align-page-rank Time: 0.23582792282104492s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 4.604219198226929s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 6.693008184432983s
string-similarity-['jaro_winkler'] Time: 0.7320289611816406s
string-similarity-['levenshtein'] Time: 6.406642913818359s
string-similarity-['jaccard:tokenizer=word'] Time: 0.1324169635772705s
normalize-scores-des_cont_jaccard Time: 0.02198505401611328s
smallest-qnode-number Time: 0.22907590866088867s
mosaic-features Time: 0.012861013412475586s
creat-singleton-feature Time: 0.12227487564086914s
vote-by-classifier Time: 0.862541913986206s
Qnodes to lookup: 3642
Qnodes from file: 3567
Outlier removal generates 55 lof-voted candidates
score-using-embedding Time: 31.502750158309937s
generate-reciprocal-rank-lof-graph-embedding-score Time: 0.4205770492553711s
compute-tf-idf-class_count Time: 33.457934856414795s
compute-tf-idf-property_count Time: 33.86403775215149s
context-match Time: 

341it [7:15:50, 114.86s/it]

align-page-rank Time: 2.310871124267578s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 14.1688871383667s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 27.21897006034851s
string-similarity-['jaro_winkler'] Time: 3.1521499156951904s
string-similarity-['levenshtein'] Time: 18.026950359344482s
string-similarity-['jaccard:tokenizer=word'] Time: 0.9855029582977295s
normalize-scores-des_cont_jaccard Time: 0.1484520435333252s
smallest-qnode-number Time: 2.767651081085205s
mosaic-features Time: 0.07256507873535156s
creat-singleton-feature Time: 1.0267040729522705s
vote-by-classifier Time: 1.1113629341125488s
Qnodes to lookup: 27602
Qnodes from file: 26841
_centroid_of_lof: Missing 1 of 266
Outlier removal generates 160 lof-voted candidates
score-using-embedding Time: 87.06353497505188s
generate-reciprocal-rank-lof-graph-embedding-score Time: 2.1906330585479736s
compute-tf-idf-class_count Time: 93.73657393455505s
compute-tf-idf-property_count Time: 95.557877

In [11]:
feature_generation(dev_candidate_path, dev_graph_embedding, dev_class_count, dev_prop_count, dev_context_path, dev_feature_path)

0it [00:00, ?it/s]

align-page-rank Time: 0.44028282165527344s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 7.408203125s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 15.187237977981567s
string-similarity-['jaro_winkler'] Time: 1.039217233657837s
string-similarity-['levenshtein'] Time: 6.074228048324585s
string-similarity-['jaccard:tokenizer=word'] Time: 0.23963499069213867s
normalize-scores-des_cont_jaccard Time: 0.0382227897644043s
smallest-qnode-number Time: 0.49535703659057617s
mosaic-features Time: 0.022897720336914062s
creat-singleton-feature Time: 0.32657718658447266s
vote-by-classifier Time: 1.6364359855651855s
Qnodes to lookup: 10982
Qnodes from file: 10654
Outlier removal generates 99 lof-voted candidates
score-using-embedding Time: 44.015238761901855s
generate-reciprocal-rank-lof-graph-embedding-score Time: 0.5955820083618164s
compute-tf-idf-class_count Time: 45.96125888824463s
compute-tf-idf-property_count Time: 46.41690492630005s
context-match Time: 59.2

1it [01:56, 116.16s/it]

align-page-rank Time: 0.08647489547729492s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 0.6216990947723389s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 1.7880198955535889s
string-similarity-['jaro_winkler'] Time: 0.17881107330322266s
string-similarity-['levenshtein'] Time: 0.6029279232025146s
string-similarity-['jaccard:tokenizer=word'] Time: 0.0405580997467041s
normalize-scores-des_cont_jaccard Time: 0.016739845275878906s
smallest-qnode-number Time: 0.1318068504333496s
mosaic-features Time: 0.007908821105957031s
creat-singleton-feature Time: 0.08887195587158203s
vote-by-classifier Time: 0.4225149154663086s
Qnodes to lookup: 3254
Qnodes from file: 3227
Outlier removal generates 8 lof-voted candidates
score-using-embedding Time: 11.774867057800293s
generate-reciprocal-rank-lof-graph-embedding-score Time: 0.290255069732666s
compute-tf-idf-class_count Time: 12.75142765045166s
compute-tf-idf-property_count Time: 13.729729652404785s
context-match Tim

2it [02:18, 60.79s/it] 

...

7it [08:41, 79.44s/it]

align-page-rank Time: 1.5963890552520752s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 9.709328174591064s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 33.90274405479431s
string-similarity-['jaro_winkler'] Time: 2.125648260116577s
string-similarity-['levenshtein'] Time: 10.14313292503357s
string-similarity-['jaccard:tokenizer=word'] Time: 0.5256590843200684s
normalize-scores-des_cont_jaccard Time: 0.09882187843322754s
smallest-qnode-number Time: 1.7649171352386475s
mosaic-features Time: 0.051434993743896484s
creat-singleton-feature Time: 0.703510046005249s
vote-by-classifier Time: 0.5056219100952148s
Qnodes to lookup: 21311
Qnodes from file: 21009
Outlier removal generates 90 lof-voted candidates
score-using-embedding Time: 75.54481887817383s
generate-reciprocal-rank-lof-graph-embedding-score Time: 1.3187470436096191s
compute-tf-idf-class_count Time: 79.4631450176239s
compute-tf-idf-property_count Time: 80.89181423187256s
context-match Time: 219.4

8it [13:50, 152.81s/it]

align-page-rank Time: 0.1799919605255127s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 3.5184550285339355s
string-similarity-['symmetric_monge_elkan:tokenizer=word'] Time: 3.9267590045928955s
string-similarity-['jaro_winkler'] Time: 0.7206149101257324s
string-similarity-['levenshtein'] Time: 3.9114248752593994s
string-similarity-['jaccard:tokenizer=word'] Time: 0.19081711769104004s
normalize-scores-des_cont_jaccard Time: 0.0239717960357666s
smallest-qnode-number Time: 0.21527671813964844s
mosaic-features Time: 0.015949249267578125s
creat-singleton-feature Time: 0.18722105026245117s
vote-by-classifier Time: 0.4609508514404297s
Qnodes to lookup: 7895
Qnodes from file: 7638
Outlier removal generates 59 lof-voted candidates
score-using-embedding Time: 21.252566814422607s
generate-reciprocal-rank-lof-graph-embedding-score Time: 0.4088780879974365s
compute-tf-idf-class_count Time: 22.600757122039795s
compute-tf-idf-property_count Time: 23.93340492248535s
context-match Tim

9it [15:45, 105.02s/it]


### Generate Training Data

In [12]:
def merge_files(args):
    datapath = args.train_path
    eval_file_names = []
    for (dirpath, dirnames, filenames) in os.walk(datapath):
        for fn in filenames:
            if "csv" not in fn:
                continue
            abs_fn = f"{dirpath}/{fn}"
            assert os.path.isfile(abs_fn)
            if os.path.getsize(abs_fn) == 0:
                continue
            eval_file_names.append(abs_fn)
    df_list = []
    for fn in eval_file_names:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        df_list.append(df)
    return pd.concat(df_list) 

def compute_normalization_factor(args, all_data):
    min_max_scaler_path = args.min_max_scaler_path
    all_data_features = all_data[features]
    scaler = MinMaxScaler()
    scaler.fit(all_data_features)
    pickle.dump(scaler, open(min_max_scaler_path, 'wb'))
    return scaler

def generate_train_data(args):
    scaler_path = args.min_max_scaler_path
    scaler = pickle.load(open(scaler_path, 'rb'))
    final_list = []
    sfeatures = copy.deepcopy(features) + ['evaluation_label']
    print(sfeatures)
    normalize_features = features
    evaluation_label = ['evaluation_label']
    positive_features_final = []
    negative_features_final = []
    for i,file in enumerate(glob.glob(args.train_path + '/*.csv')):
        file_name = file.split('/')[-1]
        print(file_name)
        if os.path.getsize(file) == 0:
                continue
        d_sample = pd.read_csv(file)
#         grouped_obj = d_sample.groupby(['row', 'column'])
        grouped_obj = d_sample.groupby(['column', 'row'])
        for cell in grouped_obj:
            cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
            pos_features = []
            neg_features = []
            a = cell[1][cell[1]['evaluation_label'] == 1]
            if a.empty:
                continue
            num_rows = 64
            pos_row = a[sfeatures].drop('evaluation_label',axis=1)
            negatives_filtered = cell[1][cell[1]['evaluation_label'] == -1]
            sorted_df = negatives_filtered.sort_values('lof-graph-embedding-score',ascending=False)
            sorted_df = sorted_df[sfeatures]
            if 0 in sorted_df['evaluation_label'].tolist():
                continue
            if sorted_df.empty:
                continue
            neg_list = []
            if num_rows < len(sorted_df):
                sorted_df = sorted_df[sorted_df['evaluation_label'] == -1]
                neg_list.append(sorted_df[:2])
                retrieval_score_df = sorted_df[2:].sort_values('retrieval_score',ascending=False)
                neg_list.append(retrieval_score_df[:2])
                pagerank_score_df = retrieval_score_df[2:].sort_values('pagerank', ascending=False)
                neg_list.append(pagerank_score_df[:2])
                class_count_score_df = pagerank_score_df[2:].sort_values('lof_class_count_tf_idf_score', ascending=False)
                neg_list.append(class_count_score_df[:2])
                prop_count_score_df = class_count_score_df[2:].sort_values('lof_property_count_tf_idf_score', ascending=False)
                neg_list.append(prop_count_score_df[:2])
                monge_elkan_score_df = prop_count_score_df[2:].sort_values('monge_elkan', ascending=False)
                neg_list.append(monge_elkan_score_df[:2])
                monge_elkan_alias_score_df = monge_elkan_score_df[2:].sort_values('monge_elkan_aliases', ascending=False)
                neg_list.append(monge_elkan_alias_score_df[:2])
                
                context_score_df = monge_elkan_alias_score_df[2:].sort_values('context_score', ascending=False)
                neg_list.append(context_score_df[:2])

                jaro_winkler_score_df = monge_elkan_alias_score_df[2:].sort_values('jaro_winkler', ascending=False)
                neg_list.append(jaro_winkler_score_df[:2])
                
                top_sample_df = jaro_winkler_score_df.sample(n=50)
                neg_list.append(top_sample_df)
                top_sample_df = pd.concat(neg_list)
                top_sample_df.drop('evaluation_label', inplace=True, axis=1)
                top_sample_arr = top_sample_df.to_numpy()

            for i in range(len(top_sample_arr)):
                neg_features.append(top_sample_arr[i])
            random.shuffle(neg_features)
            for i in range(len(top_sample_arr)):
                pos_row_sample = pos_row.sample(n=1)
                ar = pos_row_sample.to_numpy()
                for ps_ar in ar:
                    pos_features.append(ps_ar)
            positive_features_final.append(pos_features)
            negative_features_final.append(neg_features)
    print(len(positive_features_final), len(positive_features_final[37]))
    print(len(negative_features_final), len(negative_features_final[37]))
    pickle.dump(positive_features_final,open(args.pos_output,'wb'))
    pickle.dump(negative_features_final,open(args.neg_output,'wb'))


In [13]:
gen_training_data_args = Namespace(train_path=train_feature_path, pos_output=pos_output, neg_output=neg_output, 
                 min_max_scaler_path=min_max_scaler_path)
all_data = merge_files(gen_training_data_args)
scaler = compute_normalization_factor(gen_training_data_args, all_data)
generate_train_data(gen_training_data_args)


['pagerank', 'retrieval_score', 'monge_elkan', 'monge_elkan_aliases', 'des_cont_jaccard', 'jaro_winkler', 'levenshtein', 'singleton', 'num_char', 'num_tokens', 'lof_class_count_tf_idf_score', 'lof_property_count_tf_idf_score', 'lof-graph-embedding-score', 'lof-reciprocal-rank', 'context_score', 'evaluation_label']
LPK7ZPYF.csv
DPY34RCV.csv
C1N4OC0S.csv
1LD1MWA8.csv
YX7BRMMR.csv
58891288_0_1117541047012405958.csv
BBU25KIE.csv
ZX8GERJC.csv
8ZD74BO9.csv
DVFKEEPF.csv
YCXXPVD2.csv
D65TEZWN.csv
NPGBDBM4.csv
BMVA4ZET.csv
4Q6FAT24.csv
Z8HBXN1A.csv
W0ZNF869.csv
6SMY8KMG.csv
AM1UELOJ.csv
39173938_0_7916056990138658530.csv
5IXA0RAI.csv
8EFC5XVR.csv
2INS294X.csv
RW8EFPXB.csv
0KL64BZL.csv
G0QTILKH.csv
DPUA686B.csv
NUMBFEKZ.csv
UMMA6HQO.csv
ERPSWFMM.csv
ZDAZ5PQ5.csv
XF412HIL.csv
BQ36GYQE.csv
CKRLO13X.csv
JOQ4B8NW.csv
L5LFLQIN.csv
IL8WDV2Y.csv
J6SSKET3.csv
M6BLH4EG.csv
10579449_0_1681126353774891032.csv
N7ZYR1AT.csv
RMTEB8QZ.csv
ZL1S4IY0.csv
X0TEEJCK.csv
T8SL8HGK.csv
QAPGT6E5.csv
C9OCMYGQ.csv
24X84XD

### Model Definition

In [14]:
# Dataset
class T2DV2Dataset(Dataset):
    def __init__(self, pos_features, neg_features):
        self.pos_features = pos_features
        self.neg_features = neg_features
    
    def __len__(self):
        return len(self.pos_features)
    
    def __getitem__(self, idx):
        return self.pos_features[idx], self.neg_features[idx]

# Model
class PairwiseNetwork(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        #original 12x24, 24x12, 12x12, 12x1
        self.fc1 = nn.Linear(hidden_size, 2*hidden_size)
        self.fc2 = nn.Linear(2*hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, 1)
    
    def forward(self, pos_features, neg_features):
        # Positive pass
        x = F.relu(self.fc1(pos_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pos_out = torch.sigmoid(self.fc4(x))
        
        # Negative Pass
        x = F.relu(self.fc1(neg_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        neg_out = torch.sigmoid(self.fc4(x))
        
        return pos_out, neg_out
    
    def predict(self, test_feat):
        x = F.relu(self.fc1(test_feat))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        test_out = torch.sigmoid(self.fc4(x))
        return test_out

# Pairwise Loss
class PairwiseLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.m = 0
    
    def forward(self, pos_out, neg_out):
        distance = (1 - pos_out) + neg_out
        loss = torch.mean(torch.max(torch.tensor(0), distance))
        return loss

### Training

In [15]:
def generate_dataloader(positive_feat_path, negative_feat_path):
    pos_features = pickle.load(open(positive_feat_path, 'rb'))
    neg_features = pickle.load(open(negative_feat_path, 'rb'))

    pos_features_flatten = list(chain.from_iterable(pos_features))
    neg_features_flatten = list(chain.from_iterable(neg_features))

    train_dataset = T2DV2Dataset(pos_features_flatten, neg_features_flatten)
    train_dataloader = DataLoader(train_dataset, batch_size=64)
    return train_dataloader

def infer_scores(min_max_scaler_path, input_table_path, output_table_path, model):
    scaler = pickle.load(open(min_max_scaler_path, 'rb'))
    normalize_features = features
    for file in glob.glob(input_table_path + '/*.csv'):
        file_name = file.split('/')[-1]
        if os.path.getsize(file) == 0:
                continue
        if file_name != '52299421_0_4473286348258170200.csv':
            print(file_name)
            d_sample = pd.read_csv(file)
            grouped_obj = d_sample.groupby(['column', 'row'])
            new_df_list = []
            pred = []
            for cell in grouped_obj:
                cell[1][normalize_features] = scaler.transform(cell[1][normalize_features])
                sorted_df = cell[1].sort_values('lof-graph-embedding-score',ascending=False)[:64]
                sorted_df_features = sorted_df[normalize_features]
                new_df_list.append(sorted_df)
                arr = sorted_df_features.to_numpy()
                test_inp = []
                for a in arr:
                    test_inp.append(a)
                test_tensor = torch.tensor(test_inp).float()
                scores = model.predict(test_tensor)
                pred.extend(torch.squeeze(scores).tolist())
            test_df = pd.concat(new_df_list)
            test_df[final_score_column] = pred
            test_df.to_csv(f"{output_table_path}/{file_name}", index=False)

def train(args):
    if torch.cuda.is_available():
        device = torch.device('cuda')
    
    else:
        device = torch.device('cpu')
    train_dataloader = generate_dataloader(args.positive_feat_path, args.negative_feat_path)
    criterion = PairwiseLoss()
    EPOCHS = args.num_epochs
    model = PairwiseNetwork(len(features)).to(device=device)
    optimizer = Adam(model.parameters(), lr=args.lr)
    top1_max_prec = 0
    for epoch in range(EPOCHS):
        train_epoch_loss = 0
        avg_loss = 0
        model.train()
        for bid, batch in tqdm(enumerate(train_dataloader), position=0, leave=True):
            positive_feat = torch.tensor(batch[0].float())
            negative_feat = torch.tensor(batch[1].float())
            optimizer.zero_grad()
            pos_out, neg_out = model(positive_feat, negative_feat)
            loss = criterion(pos_out, neg_out)
            loss.backward()
            optimizer.step()
            train_epoch_loss += loss
        avg_loss = train_epoch_loss / bid

        # Evaluation
        model.eval()
        infer_scores(args.min_max_scaler_path, args.dev_path, args.dev_output, model)
        eval_data = merge_eval_files(args.dev_output)
        res, candidate_eval_data = parse_eval_files_stats(eval_data, final_score_column)
        top1_precision = res['num_tasks_with_model_score_top_one_accurate']/res['num_tasks_with_gt']
        if top1_precision > top1_max_prec:
            top1_max_prec = top1_precision
            model_save_name = 'epoch_{}_loss_{}_top1_{}.pth'.format(epoch, avg_loss, top1_max_prec)
            best_model_path = os.path.join(args.model_save_path, model_save_name)
            torch.save(model.state_dict(), best_model_path)
        
        print("Epoch {}, Avg Loss is {}, epoch top1 {}, max top1 {}".format(epoch, avg_loss, top1_precision, top1_max_prec))
    return best_model_path

In [16]:
def merge_eval_files(final_score_path):
    eval_file_names = []
    df_list = []
    for (dirpath, dirnames, filenames) in os.walk(final_score_path):
        for fn in filenames:
            if fn != '52299421_0_4473286348258170200.csv':
                if "csv" not in fn:
                    continue
                abs_fn = os.path.join(dirpath, fn)
                assert os.path.isfile(abs_fn)
                if os.path.getsize(abs_fn) == 0:
                    continue
                eval_file_names.append(abs_fn)
    
    for fn in eval_file_names:
        fid = fn.split('/')[-1].split('.csv')[0]
        df = pd.read_csv(fn)
        df['table_id'] = fid
        # df = df.fillna('')
        df_list.append(df)
    return pd.concat(df_list)

def parse_eval_files_stats(eval_data, method):
    res = {}
    candidate_eval_data = eval_data.groupby(['table_id', 'column', 'row'])['table_id'].count().reset_index(name="count")
    res['num_tasks'] = len(eval_data.groupby(['table_id', 'column', 'row']))
    res['num_tasks_with_gt'] = len(eval_data[pd.notna(eval_data['GT_kg_id'])].groupby(['table_id', 'column', 'row']))
    res['num_tasks_with_gt_in_candidate'] = len(eval_data[eval_data['evaluation_label'] == 1].groupby(['table_id', 'column', 'row']))
    res['num_tasks_with_singleton_candidate'] = len(candidate_eval_data[candidate_eval_data['count'] == 1].groupby(['table_id', 'column', 'row']))
    singleton_eval_data = candidate_eval_data[candidate_eval_data['count'] == 1]
    num_tasks_with_singleton_candidate_with_gt = 0
    for i, row in singleton_eval_data.iterrows():
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) == 1
        if c_e_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_singleton_candidate_with_gt += 1
    res['num_tasks_with_singleton_candidate_with_gt'] = num_tasks_with_singleton_candidate_with_gt
    num_tasks_with_graph_top_one_accurate = []
    num_tasks_with_graph_top_five_accurate = []
    num_tasks_with_graph_top_ten_accurate = []
    num_tasks_with_model_score_top_one_accurate = []
    num_tasks_with_model_score_top_five_accurate = []
    num_tasks_with_model_score_top_ten_accurate = []
    has_gt_list = []
    has_gt_in_candidate = []
    # candidate_eval_data = candidate_eval_data[:1]
    for i, row in candidate_eval_data.iterrows():
        #print(i)
        table_id, row_idx, col_idx = row['table_id'], row['row'], row['column']
        c_e_data = eval_data[(eval_data['table_id'] == table_id) & (eval_data['row'] == row_idx) & (eval_data['column'] == col_idx)]
        assert len(c_e_data) > 0
        if np.nan not in set(c_e_data['GT_kg_id']):
            has_gt_list.append(1)
        else:
            has_gt_list.append(0)
        if 1 in set(c_e_data['evaluation_label']):
            has_gt_in_candidate.append(1)
        else:
            has_gt_in_candidate.append(0)
            
        # handle graph-embedding-score
        s_data = c_e_data.sort_values(by=['lof-graph-embedding-score'], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_graph_top_one_accurate.append(1)
        else:
            num_tasks_with_graph_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_graph_top_five_accurate.append(1)
        else:
            num_tasks_with_graph_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_graph_top_ten_accurate.append(1)
        else:
            num_tasks_with_graph_top_ten_accurate.append(0)
        
        #rank on model score
        s_data = c_e_data.sort_values(by=[method], ascending=False)
        if s_data.iloc[0]['evaluation_label'] == 1:
            num_tasks_with_model_score_top_one_accurate.append(1)
        else:
            num_tasks_with_model_score_top_one_accurate.append(0)
        if 1 in set(s_data.iloc[0:5]['evaluation_label']):
            num_tasks_with_model_score_top_five_accurate.append(1)
        else:
            num_tasks_with_model_score_top_five_accurate.append(0)
        if 1 in set(s_data.iloc[0:10]['evaluation_label']):
            num_tasks_with_model_score_top_ten_accurate.append(1)
        else:
            num_tasks_with_model_score_top_ten_accurate.append(0)
            
        cf_e_data = c_e_data.copy()
        cf_e_data['lof-graph-embedding-score'] = cf_e_data['lof-graph-embedding-score'].replace(np.nan, 0)
        cf_e_data[method] = cf_e_data[method].replace(np.nan, 0)

    candidate_eval_data['lof-graph_top_one_accurate'] = num_tasks_with_graph_top_one_accurate
    candidate_eval_data['lof-graph_top_five_accurate'] = num_tasks_with_graph_top_five_accurate
    candidate_eval_data['lof-graph_top_ten_accurate'] = num_tasks_with_graph_top_five_accurate
    candidate_eval_data['model_top_one_accurate'] = num_tasks_with_model_score_top_one_accurate
    candidate_eval_data['model_top_five_accurate'] = num_tasks_with_model_score_top_five_accurate
    candidate_eval_data['model_top_ten_accurate'] = num_tasks_with_model_score_top_ten_accurate
    candidate_eval_data['has_gt'] = has_gt_list
    candidate_eval_data['has_gt_in_candidate'] = has_gt_in_candidate
    res['num_tasks_with_graph_top_one_accurate'] = sum(num_tasks_with_graph_top_one_accurate)
    res['num_tasks_with_graph_top_five_accurate'] = sum(num_tasks_with_graph_top_five_accurate)
    res['num_tasks_with_graph_top_ten_accurate'] = sum(num_tasks_with_graph_top_ten_accurate)
    res['num_tasks_with_model_score_top_one_accurate'] = sum(num_tasks_with_model_score_top_one_accurate)
    res['num_tasks_with_model_score_top_five_accurate'] = sum(num_tasks_with_model_score_top_five_accurate)
    res['num_tasks_with_model_score_top_ten_accurate'] = sum(num_tasks_with_model_score_top_ten_accurate)
    return res, candidate_eval_data

In [17]:
training_args = Namespace(num_epochs=20, lr=0.001, positive_feat_path=pos_output, negative_feat_path=neg_output,
                         dev_path=dev_feature_path, dev_output=dev_output_predictions,
                         model_save_path=model_save_path, min_max_scaler_path=min_max_scaler_path)

In [18]:
## Call Training
best_model_path = train(training_args)

0it [00:00, ?it/s]<ipython-input-15-7a187275f3b9>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
<ipython-input-15-7a187275f3b9>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
18278it [00:30, 602.41it/s]


39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv


0it [00:00, ?it/s]<ipython-input-15-7a187275f3b9>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
<ipython-input-15-7a187275f3b9>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
106it [00:00, 524.60it/s]

Epoch 0, Avg Loss is 0.12883563339710236, epoch top1 0.6200564971751412, max top1 0.6200564971751412


18278it [00:30, 601.79it/s]


...

39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv


0it [00:00, ?it/s]<ipython-input-15-7a187275f3b9>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_feat = torch.tensor(batch[0].float())
<ipython-input-15-7a187275f3b9>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negative_feat = torch.tensor(batch[1].float())
52it [00:00, 516.10it/s]

Epoch 18, Avg Loss is 0.11798889935016632, epoch top1 0.5805084745762712, max top1 0.8403954802259888


18278it [00:31, 581.02it/s]


39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv
Epoch 19, Avg Loss is 0.11301832646131516, epoch top1 0.8248587570621468, max top1 0.8403954802259888


In [19]:
best_model_path

'/Users/amandeep/Github/table-linker/data/SemTabR4_T2dv2/table-linker/dev-output/v12/saved_models/epoch_12_loss_0.1164768785238266_top1_0.8403954802259888.pth'

## Dev Prediction

In [20]:
def dev_prediction(dev_feature_path, dev_predictions_top_k, saved_model, output_column, min_max_scaler_path, k=5):
    for file in glob.glob(dev_feature_path + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        feature_str =  ",".join(features)
        if os.path.getsize(file) == 0:
                    continue
        # location where the output generated by the predictions wil be stored.
        dev_output = f"{dev_predictions_top_k}/{filename}"
        !tl predict-using-model $file -o $output_column \
            --features $feature_str \
            --ranking-model $saved_model \
            --normalization-factor $min_max_scaler_path \
            / get-kg-links -c $output_column -k $k --k-rows \
            > $dev_output

In [21]:
def add_color(dev_predictions_top_k, dev_colorized_path, score_column, k=5):
    for file in glob.glob(dev_predictions_top_k + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
                
        dev_color_file = f"{dev_colorized_path}/{filename.strip('.csv')}.xlsx"
        !tl add-color $file -c "$score_column,evaluation_label" -k $k --output $dev_color_file

In [22]:
def compute_metrics(dev_predictions_top_k, dev_predictions_metrics, score_column, k=5):
    df_list = []
    for file in glob.glob(dev_predictions_top_k + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
                
        dev_metrics_file = f"{dev_predictions_metrics}/{filename}"
        !tl metrics $file -k $k -c $score_column --tag $filename> $dev_metrics_file
        df_list.append(pd.read_csv(dev_metrics_file))
    return pd.concat(df_list)

In [23]:
dev_prediction(dev_feature_path, dev_predictions_top_k, best_model_path, final_score_column, min_max_scaler_path, k=200)

39759273_0_1427898308030295194.csv
predict-using-model Time: 1.1223816871643066s
get-kg-links-siamese_prediction Time: 0.9894561767578125s
45073662_0_3179937335063201739.csv
predict-using-model Time: 1.8837628364562988s
get-kg-links-siamese_prediction Time: 0.2691960334777832s
29414811_2_4773219892816395776.csv
predict-using-model Time: 1.1233410835266113s
get-kg-links-siamese_prediction Time: 0.21491312980651855s
84575189_0_6365692015941409487.csv
predict-using-model Time: 1.5535120964050293s
get-kg-links-siamese_prediction Time: 1.0447988510131836s
14380604_4_3329235705746762392.csv
predict-using-model Time: 0.9142780303955078s
get-kg-links-siamese_prediction Time: 0.2114419937133789s
52299421_0_4473286348258170200.csv
predict-using-model Time: 1.4488623142242432s
get-kg-links-siamese_prediction Time: 1.1445989608764648s
50270082_0_444360818941411589.csv
predict-using-model Time: 1.6169407367706299s
get-kg-links-siamese_prediction Time: 1.736905813217163s
28086084_0_31276605309899167

In [24]:
metrics_df = compute_metrics(dev_predictions_top_k, dev_metrics_path, final_score_column, k=200)

39759273_0_1427898308030295194.csv
metrics Time: 3.344841957092285s
45073662_0_3179937335063201739.csv
metrics Time: 0.9158201217651367s
29414811_2_4773219892816395776.csv
metrics Time: 0.7286691665649414s
84575189_0_6365692015941409487.csv
metrics Time: 3.2659289836883545s
14380604_4_3329235705746762392.csv
metrics Time: 0.828848123550415s
52299421_0_4473286348258170200.csv
metrics Time: 4.336864948272705s
50270082_0_444360818941411589.csv
metrics Time: 5.679181814193726s
28086084_0_3127660530989916727.csv
metrics Time: 7.728912830352783s
14067031_0_559833072073397908.csv
metrics Time: 2.216524124145508s


In [25]:
metrics_df

,k,f1,precision,recall,tag
0,200,0.994975,0.990000,1.000000,39759273_0_1427898308030295194.csv
0,200,1.000000,1.000000,1.000000,45073662_0_3179937335063201739.csv
0,200,0.976744,0.954545,1.000000,29414811_2_4773219892816395776.csv
0,200,0.898876,0.816327,1.000000,84575189_0_6365692015941409487.csv
0,200,1.000000,1.000000,1.000000,14380604_4_3329235705746762392.csv
0,200,0.978022,0.978022,0.978022,52299421_0_4473286348258170200.csv
0,200,0.990991,0.982143,1.000000,50270082_0_444360818941411589.csv
0,200,0.854880,0.768182,0.963636,28086084_0_3127660530989916727.csv
0,200,0.981132,0.981132,0.981132,14067031_0_559833072073397908.csv


In [26]:
metrics_df['recall'].mean()

0.9914211574588934

In [27]:
metrics_df.to_csv(f"{dev_metrics_path}/metrics_200.csv", index=False)

In [28]:
metrics_df = compute_metrics(dev_predictions_top_k, dev_metrics_path, final_score_column, k=1)

39759273_0_1427898308030295194.csv
metrics Time: 3.5537333488464355s
45073662_0_3179937335063201739.csv
metrics Time: 0.9060759544372559s
29414811_2_4773219892816395776.csv
metrics Time: 0.6573503017425537s
84575189_0_6365692015941409487.csv
metrics Time: 3.3251709938049316s
14380604_4_3329235705746762392.csv
metrics Time: 0.7085330486297607s
52299421_0_4473286348258170200.csv
metrics Time: 4.736017227172852s
50270082_0_444360818941411589.csv
metrics Time: 5.278078079223633s
28086084_0_3127660530989916727.csv
metrics Time: 7.827936887741089s
14067031_0_559833072073397908.csv
metrics Time: 2.3459389209747314s


In [29]:
metrics_df

,k,f1,precision,recall,tag
0,1,0.897017,0.990000,0.820000,39759273_0_1427898308030295194.csv
0,1,0.920000,1.000000,0.851852,45073662_0_3179937335063201739.csv
0,1,0.526646,0.954545,0.363636,29414811_2_4773219892816395776.csv
0,1,0.800728,0.816327,0.785714,84575189_0_6365692015941409487.csv
0,1,0.947368,1.000000,0.900000,14380604_4_3329235705746762392.csv
0,1,0.894532,0.978022,0.824176,52299421_0_4473286348258170200.csv
0,1,0.948276,0.982143,0.916667,50270082_0_444360818941411589.csv
0,1,0.756648,0.768182,0.745455,28086084_0_3127660530989916727.csv
0,1,0.941887,0.981132,0.905660,14067031_0_559833072073397908.csv


In [30]:
metrics_df['f1'].mean()

0.848122356254435

In [31]:
metrics_df.to_csv(f"{dev_metrics_path}/metrics_1.csv", index=False)

In [32]:
add_color(dev_predictions_top_k, dev_colorized_path, final_score_column)

39759273_0_1427898308030295194.csv
add-color Time: 2.080474853515625s
45073662_0_3179937335063201739.csv
add-color Time: 0.4650721549987793s
29414811_2_4773219892816395776.csv
add-color Time: 0.3892197608947754s
84575189_0_6365692015941409487.csv
add-color Time: 1.7831270694732666s
14380604_4_3329235705746762392.csv
add-color Time: 0.39830708503723145s
52299421_0_4473286348258170200.csv
add-color Time: 2.531569719314575s
50270082_0_444360818941411589.csv
add-color Time: 2.867861032485962s
28086084_0_3127660530989916727.csv
add-color Time: 4.324522018432617s
14067031_0_559833072073397908.csv
add-color Time: 1.5757498741149902s


In [33]:
def find_missing_correct_candidates(candidates_path, missing_correct_candidates_path):
     for file in tqdm(glob.glob(candidates_path + '/*.csv')):
        filename = file.split("/")[-1]
        if os.path.getsize(file) == 0:
                    continue
        missing_file = f"{missing_correct_candidates_path}/{filename}"
        !tl check-candidates "$file" > "$missing_file"

In [34]:
find_missing_correct_candidates(dev_candidate_path, dev_missing_candidates_path)

  0%|          | 0/9 [00:00<?, ?it/s]

check-candidates Time: 0.020513057708740234s


 11%|█         | 1/9 [00:02<00:23,  3.00s/it]

check-candidates Time: 0.007582902908325195s


 22%|██▏       | 2/9 [00:06<00:21,  3.01s/it]

...

 78%|███████▊  | 7/9 [00:14<00:03,  1.69s/it]

check-candidates Time: 0.04585003852844238s


 89%|████████▉ | 8/9 [00:15<00:01,  1.55s/it]

check-candidates Time: 0.015259027481079102s


100%|██████████| 9/9 [00:17<00:00,  1.94s/it]


In [35]:
def concat_files(files_path):
    df_list = []
    for file in glob.glob(files_path + '/*.csv'):
        filename = file.split("/")[-1]
        print(filename)
        if os.path.getsize(file) == 0:
                    continue
        df = pd.read_csv(file)
        df['filename'] = filename
        df_list.append(df)
    return pd.concat(df_list)

In [36]:
missing_df = concat_files(dev_missing_candidates_path)

39759273_0_1427898308030295194.csv
45073662_0_3179937335063201739.csv
29414811_2_4773219892816395776.csv
84575189_0_6365692015941409487.csv
14380604_4_3329235705746762392.csv
52299421_0_4473286348258170200.csv
50270082_0_444360818941411589.csv
28086084_0_3127660530989916727.csv
14067031_0_559833072073397908.csv


In [37]:
missing_df

,column,row,label,context,GT_kg_id,GT_kg_label,filename
0,1,50,World,"51|$6,500|4.00%|$38,000.0|25.00%|5,926,466,814...",Q2277,Roman Empire,52299421_0_4473286348258170200.csv
0,0,38,BRIGID of Ireland,--|--|--|Yes|--,Q18527449,Dar Lugdach,28086084_0_3127660530989916727.csv
1,0,39,Brigid of Kildare,--|--|--|Yes|--,Q18527449,Dar Lugdach,28086084_0_3127660530989916727.csv
2,0,52,Charlie,--|--|--|Yes|--,Q1199757,Dish Network,28086084_0_3127660530989916727.csv
3,0,59,"Cyril, Apostle to the Slavs",??|??|Yes|??|--,Q239925,Saints Cyril and Methodius,28086084_0_3127660530989916727.csv
4,0,138,Louis,--|--|--|Yes|--,Q88927,Louis de Montfort,28086084_0_3127660530989916727.csv
5,0,156,Matthias,Yes|Yes|Yes|Yes|--,Q43600,Matthew the Apostle,28086084_0_3127660530989916727.csv
6,0,175,Peter of Alexandria,--|Yes|Yes|Yes|--,Q20815727,Pope Peter of Alexandria,28086084_0_3127660530989916727.csv
0,0,24,Kanya,"Swahili, English|199.4|Dec 1963|Musalia Mudava...",Q114,Kenya,14067031_0_559833072073397908.csv


In [ ]:
missing_df.to_csv(f"{dev_missing_candidates_path}/missing_concatenated.csv", index=False)

In [ ]:
find_missing_correct_candidates(train_candidates_path, train_missing_candidates_path)

In [ ]:
train_missing_df = concat_files(train_missing_candidates_path)

In [ ]:
len(train_missing_df)

In [ ]:
train_missing_df

In [ ]:
train_missing_df.to_csv(f"{train_missing_candidates_path}/missing_concatenated.csv", index=False)